#📌 Extracão

🟦 1. Importar as bibliotecas necessárias

In [16]:
# Bibliotecas
import pandas as pd
import requests


🟦 2. Definir o link da API e fazer a requisição

In [17]:
# URL da API com os dados no formato JSON
url = 'https://raw.githubusercontent.com/alura-cursos/challenge2-data-science/refs/heads/main/TelecomX_Data.json'

# Fazendo a requisição HTTP para obter os dados da API
response = requests.get(url)

# Verificando se a requisição foi bem-sucedida (status code 200)
if response.status_code == 200:
    print("✅ Dados carregados com sucesso!")
else:
    print(f"❌ Falha ao carregar os dados. Código de status: {response.status_code}")


✅ Dados carregados com sucesso!


🟦 3. Converter os dados JSON em um DataFrame

In [18]:
# Convertendo o conteúdo da resposta em JSON
dados_json = response.json()

# Criando um DataFrame a partir dos dados JSON
df = pd.DataFrame(dados_json)

# Exibindo as 5 primeiras linhas do DataFrame para conferência
df.head()


,customerID,Churn,customer,phone,internet,account
0,0002-ORFBO,No,"{'gender': 'Female', 'SeniorCitizen': 0, 'Part...","{'PhoneService': 'Yes', 'MultipleLines': 'No'}","{'InternetService': 'DSL', 'OnlineSecurity': '...","{'Contract': 'One year', 'PaperlessBilling': '..."
1,0003-MKNFE,No,"{'gender': 'Male', 'SeniorCitizen': 0, 'Partne...","{'PhoneService': 'Yes', 'MultipleLines': 'Yes'}","{'InternetService': 'DSL', 'OnlineSecurity': '...","{'Contract': 'Month-to-month', 'PaperlessBilli..."
2,0004-TLHLJ,Yes,"{'gender': 'Male', 'SeniorCitizen': 0, 'Partne...","{'PhoneService': 'Yes', 'MultipleLines': 'No'}","{'InternetService': 'Fiber optic', 'OnlineSecu...","{'Contract': 'Month-to-month', 'PaperlessBilli..."
3,0011-IGKFF,Yes,"{'gender': 'Male', 'SeniorCitizen': 1, 'Partne...","{'PhoneService': 'Yes', 'MultipleLines': 'No'}","{'InternetService': 'Fiber optic', 'OnlineSecu...","{'Contract': 'Month-to-month', 'PaperlessBilli..."
4,0013-EXCHZ,Yes,"{'gender': 'Female', 'SeniorCitizen': 1, 'Part...","{'PhoneService': 'Yes', 'MultipleLines': 'No'}","{'InternetService': 'Fiber optic', 'OnlineSecu...","{'Contract': 'Month-to-month', 'PaperlessBilli..."


---
As colunas customer, phone, internet e account estão armazenando dicionários (dict) como valores — ou seja, cada célula dessas colunas contém informações aninhadas que precisam ser "explodidas" (ou melhor, normalizadas ou expandida coluna a coluna).

---

🟦 4. Expandir (explodir) as colunas aninhadas

In [19]:
# Vamos fazer uma cópia do dataframe original, para não perdermos os dados iniciais
df_exp = df.copy()

# Explodindo cada coluna que contém um dicionário em novas colunas
# Usamos o método pd.json_normalize para "desempacotar" os dados de cada célula
# e depois concatenamos os resultados no dataframe original

# Coluna: customer
customer_df = pd.json_normalize(df_exp['customer'])
df_exp = pd.concat([df_exp.drop(columns=['customer']), customer_df], axis=1)

# Coluna: phone
phone_df = pd.json_normalize(df_exp['phone'])
df_exp = pd.concat([df_exp.drop(columns=['phone']), phone_df], axis=1)

# Coluna: internet
internet_df = pd.json_normalize(df_exp['internet'])
df_exp = pd.concat([df_exp.drop(columns=['internet']), internet_df], axis=1)

# Coluna: account
account_df = pd.json_normalize(df_exp['account'])
df_exp = pd.concat([df_exp.drop(columns=['account']), account_df], axis=1)

# Exibindo as primeiras linhas do novo DataFrame com colunas planificadas
df_exp.head()


,customerID,Churn,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,...,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,Charges.Monthly,Charges.Total
0,0002-ORFBO,No,Female,0,Yes,Yes,9,Yes,No,DSL,...,Yes,No,Yes,Yes,No,One year,Yes,Mailed check,65.6,593.3
1,0003-MKNFE,No,Male,0,No,No,9,Yes,Yes,DSL,...,No,No,No,No,Yes,Month-to-month,No,Mailed check,59.9,542.4
2,0004-TLHLJ,Yes,Male,0,No,No,4,Yes,No,Fiber optic,...,No,Yes,No,No,No,Month-to-month,Yes,Electronic check,73.9,280.85
3,0011-IGKFF,Yes,Male,1,Yes,No,13,Yes,No,Fiber optic,...,Yes,Yes,No,Yes,Yes,Month-to-month,Yes,Electronic check,98.0,1237.85
4,0013-EXCHZ,Yes,Female,1,Yes,No,3,Yes,No,Fiber optic,...,No,No,Yes,Yes,No,Month-to-month,Yes,Mailed check,83.9,267.4


🟦 5. Verificar os dados do dataframe

In [20]:
# Verificando as informações do dataframe
df_exp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7267 entries, 0 to 7266
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   customerID        7267 non-null   object 
 1   Churn             7267 non-null   object 
 2   gender            7267 non-null   object 
 3   SeniorCitizen     7267 non-null   int64  
 4   Partner           7267 non-null   object 
 5   Dependents        7267 non-null   object 
 6   tenure            7267 non-null   int64  
 7   PhoneService      7267 non-null   object 
 8   MultipleLines     7267 non-null   object 
 9   InternetService   7267 non-null   object 
 10  OnlineSecurity    7267 non-null   object 
 11  OnlineBackup      7267 non-null   object 
 12  DeviceProtection  7267 non-null   object 
 13  TechSupport       7267 non-null   object 
 14  StreamingTV       7267 non-null   object 
 15  StreamingMovies   7267 non-null   object 
 16  Contract          7267 non-null   object 


🟦 6. Salvar o DataFrame em um arquivo CSV

In [21]:
# Salvando o DataFrame em um arquivo CSV com separador padrão (vírgula)
# O arquivo será salvo no diretório raiz do Google Colab (pode ser baixado em seguida)
df_exp.to_csv('dados_telecomx.csv', index=False)

print("💾 Arquivo 'dados_telecomx.csv' salvo com sucesso!")


💾 Arquivo 'dados_telecomx.csv' salvo com sucesso!


#🔧 Transformação

#📊 Carga e análise

#📄Relatorio Final